In [2]:
import carla
import random

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
my_map = world.get_map()
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

Own code starts here.

In [4]:
my_car_bp = world.get_blueprint_library().find('vehicle.tesla.model3')

In [21]:
ego = world.spawn_actor(blueprint=my_car_bp, transform=random.choice(spawn_points))
collision_sensor_bp = world.get_blueprint_library().find('sensor.other.collision')
cs_transform = carla.Transform(carla.Location(x=0, y=0, z=2))
cs_sensor = world.spawn_actor(
    collision_sensor_bp,
    transform=cs_transform,
    attach_to=ego
)
def on_collision(event):
    actor_type = event.other_actor.type_id
    impulse = event.normal_impulse
    # print(f"Collision with {actor_type} at impulse {impulse}")

cs_sensor.listen(on_collision)
ego.set_autopilot(True)

In [27]:
# Helper functions
def drive_vehicle(vehicle, throttle=0.5, steer=0.0):
    """Apply throttle and steer to the vehicle."""
    control = carla.VehicleControl()
    control.throttle = throttle
    control.steer = steer
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    """Stop the vehicle."""
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)

In [30]:
ego.set_autopilot(False)
stop_vehicle(ego)

In [28]:
drive_vehicle(ego)

In [11]:
import time

In [19]:
collision_detected = False
# Modify the existing code to include better handling for reversing
try:
    while True:
        if collision_detected:
            # Stop the vehicle immediately
            print("Collision detected. Stopping the vehicle.")
            stop_vehicle(ego)
            time.sleep(1)  # Short pause after collision
            
            # Attempt to reverse
            print("Reversing for 4 seconds...")
            start_time = time.time()
            while time.time() - start_time < 4:
                if collision_detected:  # Check for additional collisions while reversing
                    print("Collision while reversing. Stopping.")
                    stop_vehicle(ego)
                    break
                ego.apply_control(carla.VehicleControl(throttle=0.6, reverse=True))  # Increase throttle to ensure movement
            
            # Stop and decide a random turn
            print("Deciding on a random turn...")
            stop_vehicle(ego)
            turn_direction = random.choice([-1, 1])  # -1 for left, 1 for right
            print(f"Turning {'left' if turn_direction == -1 else 'right'} for 5 seconds...")
            
            # Turn for 5 seconds
            start_time = time.time()
            while time.time() - start_time < 5:
                if collision_detected:  # Check for additional collisions
                    print("Collision while turning. Stopping.")
                    stop_vehicle(ego)
                    break
                ego.apply_control(carla.VehicleControl(throttle=0.5, steer=turn_direction * 0.5))
            
            # Stop after turning
            print("Resuming straight drive.")
            stop_vehicle(ego)
            collision_detected = False  # Reset collision flag

        else:
            # Drive straight
            drive_vehicle(ego, throttle=0.5, steer=0.0)

finally:
    # Cleanup
    print("Cleaning up actors.")
    cs_sensor.destroy()
    ego.destroy()


Cleaning up actors.


KeyboardInterrupt: 